In [ ]:
import os
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import random
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
import plotly.graph_objects as go

# === Settings
output_dir = r"C:\Users\vishn\OneDrive\Documents\Machine Learning\ABC_ANN"
os.makedirs(output_dir, exist_ok=True)

# === Reproducibility
def set_seed(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)

set_seed(42)

# === Load Data
df = pd.read_csv(r"resources/data.csv")
X = df.drop(columns=["FoS", "SeismicFoS"]).values
y = df["FoS"].values  # Change to df["SeismicFoS"].values for seismic model

# === Preprocessing
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32).view(-1, 1)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32).view(-1, 1)

train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

# === ANN Model
class ANN(nn.Module):
    def __init__(self, input_dim, h1, h2, h3):
        super(ANN, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(input_dim, h1), nn.ReLU(),
            nn.Linear(h1, h2), nn.ReLU(),
            nn.Linear(h2, h3), nn.ReLU(),
            nn.Linear(h3, 1)
        )
    def forward(self, x):
        return self.model(x)

# === Evaluation Function
def evaluate_model(h1, h2, h3, lr):
    model = ANN(X_train.shape[1], h1, h2, h3)
    criterion = nn.MSELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    for epoch in range(100):
        model.train()
        for xb, yb in train_loader:
            optimizer.zero_grad()
            pred = model(xb)
            loss = criterion(pred, yb)
            loss.backward()
            optimizer.step()
    model.eval()
    with torch.no_grad():
        y_train_pred = model(X_train_tensor).numpy()
        y_test_pred = model(X_test_tensor).numpy()

    metrics = {
        "r2_train": r2_score(y_train, y_train_pred),
        "r2_test": r2_score(y_test, y_test_pred),
        "rmse": np.sqrt(mean_squared_error(y_test, y_test_pred)),
        "mae": mean_absolute_error(y_test, y_test_pred),
        "mse": mean_squared_error(y_test, y_test_pred),
        "rmse_train": np.sqrt(mean_squared_error(y_train, y_train_pred)),
        "mae_train": mean_absolute_error(y_train, y_train_pred),
        "mse_train": mean_squared_error(y_train, y_train_pred)
    }
    return metrics

# === ABC Parameters
num_food_sources = 10
limit = 5
cycles = 10

# === Initialize Food Sources
def init_food_sources():
    return [{
        "h1": random.randint(16, 128),
        "h2": random.randint(16, 128),
        "h3": random.randint(16, 128),
        "lr": round(random.uniform(0.0001, 0.01), 5),
        "trial": 0
    } for _ in range(num_food_sources)]

food_sources = init_food_sources()
best_solution = None
best_score = -np.inf
history = []

# === ABC Optimization Loop
for cycle in range(cycles):
    print(f"\n🔄 Cycle {cycle+1}")
    for fs in food_sources:
        metrics = evaluate_model(fs["h1"], fs["h2"], fs["h3"], fs["lr"])
        fs.update(metrics)
        if metrics["r2_test"] > best_score:
            best_score = metrics["r2_test"]
            best_solution = fs.copy()
    # Onlooker Bees
    fitnesses = [fs["r2_test"] for fs in food_sources]
    total_fitness = sum(fitnesses)
    probs = [f / total_fitness for f in fitnesses]
    new_sources = []
    for _ in range(num_food_sources):
        selected = np.random.choice(food_sources, p=probs)
        candidate = selected.copy()
        idx = random.choice(["h1", "h2", "h3", "lr"])
        if idx != "lr":
            candidate[idx] = random.randint(16, 128)
        else:
            candidate[idx] = round(random.uniform(0.0001, 0.01), 5)
        new_sources.append(candidate)
    food_sources = new_sources

    # Scout Bees
    for fs in food_sources:
        fs["trial"] += 1
        if fs["trial"] > limit:
            fs.update({
                "h1": random.randint(16, 128),
                "h2": random.randint(16, 128),
                "h3": random.randint(16, 128),
                "lr": round(random.uniform(0.0001, 0.01), 5),
                "trial": 0
            })

    print(f"🐝 Best so far: ({best_solution['h1']}, {best_solution['h2']}, {best_solution['h3']}) | "
          f"lr={best_solution['lr']} | R² Test = {best_solution['r2_test']:.6f}")
    history.append((cycle+1, best_solution['r2_test']))

# === Final Results
print("\n📊 Final ABC-ANN Evaluation for FoS")
print(f"Best Hyperparameters: ({best_solution['h1']}, {best_solution['h2']}, {best_solution['h3']}, {best_solution['lr']})")
print(f"R² (Train): {best_solution['r2_train']:.6f}")
print(f"R² (Test):  {best_solution['r2_test']:.6f}")
print(f"RMSE (Test): {best_solution['rmse']:.6f}")
print(f"MAE (Test):  {best_solution['mae']:.6f}")
print(f"MSE (Test):  {best_solution['mse']:.6f}")
print(f"RMSE (Train): {best_solution['rmse_train']:.6f}")
print(f"MAE (Train):  {best_solution['mae_train']:.6f}")
print(f"MSE (Train):  {best_solution['mse_train']:.6f}")

# === Radar Plot
metrics_labels = ["R2_Train", "R2_Test", "RMSE_Test", "MAE_Test", "MSE_Test", "RMSE_Train", "MAE_Train", "MSE_Train"]
metrics_values = [
    best_solution["r2_train"],
    best_solution["r2_test"],
    best_solution["rmse"],
    best_solution["mae"],
    best_solution["mse"],
    best_solution["rmse_train"],
    best_solution["mae_train"],
    best_solution["mse_train"]
]

# Normalize values for radar readability
max_vals = [1.0, 1.0, max(metrics_values[2:]), max(metrics_values[2:]), max(metrics_values[2:]),
            max(metrics_values[5:]), max(metrics_values[5:]), max(metrics_values[5:])]
norm_values = [v / m for v, m in zip(metrics_values, max_vals)]

# Create radar plot
fig = go.Figure()
fig.add_trace(go.Scatterpolar(
    r=norm_values + [norm_values[0]],
    theta=metrics_labels + [metrics_labels[0]],
    fill='toself',
    name='ABC-ANN FoS',
    line=dict(color='royalblue')
))
fig.update_layout(
    title="ABC-ANN Performance Radar Plot (FoS)",
    polar=dict(radialaxis=dict(visible=True, range=[0, 1])),
    showlegend=True
)

# Save plot
plot_path = os.path.join(output_dir, "ABC_ANN_FoS_RadarPlot.html")
fig.write_html(plot_path)
print(f"\n📁 Radar Plot saved to: {plot_path}")



🔄 Cycle 1
🐝 Best so far: (102, 110, 85) | lr=0.00096 | R² Test = 0.915230

🔄 Cycle 2
🐝 Best so far: (64, 20, 19) | lr=0.00103 | R² Test = 0.923560

🔄 Cycle 3
🐝 Best so far: (69, 44, 26) | lr=0.00593 | R² Test = 0.927838

🔄 Cycle 4
🐝 Best so far: (69, 44, 26) | lr=0.00593 | R² Test = 0.927838

🔄 Cycle 5
🐝 Best so far: (69, 44, 26) | lr=0.00593 | R² Test = 0.927838

🔄 Cycle 6
🐝 Best so far: (69, 44, 26) | lr=0.00593 | R² Test = 0.927838

🔄 Cycle 7
🐝 Best so far: (69, 44, 26) | lr=0.00593 | R² Test = 0.927838

🔄 Cycle 8
🐝 Best so far: (69, 44, 26) | lr=0.00593 | R² Test = 0.927838

🔄 Cycle 9
🐝 Best so far: (69, 44, 26) | lr=0.00593 | R² Test = 0.927838

🔄 Cycle 10
🐝 Best so far: (69, 44, 26) | lr=0.00593 | R² Test = 0.927838

📊 Final ABC-ANN Evaluation for FoS
Best Hyperparameters: (69, 44, 26, 0.00593)
R² (Train): 0.995709
R² (Test):  0.927838
RMSE (Test): 0.341743
MAE (Test):  0.225108
MSE (Test):  0.116788
RMSE (Train): 0.075535
MAE (Train):  0.055359
MSE (Train):  0.005706

📁 Radar P

In [2]:
#SeismicFoS
import os
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import random
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
import plotly.graph_objects as go

# === Settings
output_dir = r"C:\Users\vishn\OneDrive\Documents\Machine Learning\ABC_ANN\Seismic"
os.makedirs(output_dir, exist_ok=True)

# === Reproducibility
def set_seed(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)

set_seed(42)

# === Load Data
df = pd.read_csv(r"C:\Users\vishn\OneDrive\Documents\Machine Learning\Vishnu_phd.csv")
X = df.drop(columns=["FoS", "SeismicFoS"]).values
y = df["SeismicFoS"].values  # 🔁 Changed to SeismicFoS

# === Preprocessing
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32).view(-1, 1)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32).view(-1, 1)

train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

# === ANN Model
class ANN(nn.Module):
    def __init__(self, input_dim, h1, h2, h3):
        super(ANN, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(input_dim, h1), nn.ReLU(),
            nn.Linear(h1, h2), nn.ReLU(),
            nn.Linear(h2, h3), nn.ReLU(),
            nn.Linear(h3, 1)
        )
    def forward(self, x):
        return self.model(x)

# === Evaluation Function
def evaluate_model(h1, h2, h3, lr):
    model = ANN(X_train.shape[1], h1, h2, h3)
    criterion = nn.MSELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    for epoch in range(100):
        model.train()
        for xb, yb in train_loader:
            optimizer.zero_grad()
            pred = model(xb)
            loss = criterion(pred, yb)
            loss.backward()
            optimizer.step()
    model.eval()
    with torch.no_grad():
        y_train_pred = model(X_train_tensor).numpy()
        y_test_pred = model(X_test_tensor).numpy()

    metrics = {
        "r2_train": r2_score(y_train, y_train_pred),
        "r2_test": r2_score(y_test, y_test_pred),
        "rmse": np.sqrt(mean_squared_error(y_test, y_test_pred)),
        "mae": mean_absolute_error(y_test, y_test_pred),
        "mse": mean_squared_error(y_test, y_test_pred),
        "rmse_train": np.sqrt(mean_squared_error(y_train, y_train_pred)),
        "mae_train": mean_absolute_error(y_train, y_train_pred),
        "mse_train": mean_squared_error(y_train, y_train_pred)
    }
    return metrics

# === ABC Parameters
num_food_sources = 10
limit = 5
cycles = 10

# === Initialize Food Sources
def init_food_sources():
    return [{
        "h1": random.randint(16, 128),
        "h2": random.randint(16, 128),
        "h3": random.randint(16, 128),
        "lr": round(random.uniform(0.0001, 0.01), 5),
        "trial": 0
    } for _ in range(num_food_sources)]

food_sources = init_food_sources()
best_solution = None
best_score = -np.inf
history = []

# === ABC Optimization Loop
for cycle in range(cycles):
    print(f"\n🔄 Cycle {cycle+1}")
    for fs in food_sources:
        metrics = evaluate_model(fs["h1"], fs["h2"], fs["h3"], fs["lr"])
        fs.update(metrics)
        if metrics["r2_test"] > best_score:
            best_score = metrics["r2_test"]
            best_solution = fs.copy()

    # Onlooker Bees
    fitnesses = [fs["r2_test"] for fs in food_sources]
    total_fitness = sum(fitnesses)
    probs = [f / total_fitness for f in fitnesses]
    new_sources = []
    for _ in range(num_food_sources):
        selected = np.random.choice(food_sources, p=probs)
        candidate = selected.copy()
        idx = random.choice(["h1", "h2", "h3", "lr"])
        if idx != "lr":
            candidate[idx] = random.randint(16, 128)
        else:
            candidate[idx] = round(random.uniform(0.0001, 0.01), 5)
        new_sources.append(candidate)
    food_sources = new_sources

    # Scout Bees
    for fs in food_sources:
        fs["trial"] += 1
        if fs["trial"] > limit:
            fs.update({
                "h1": random.randint(16, 128),
                "h2": random.randint(16, 128),
                "h3": random.randint(16, 128),
                "lr": round(random.uniform(0.0001, 0.01), 5),
                "trial": 0
            })

    print(f"🐝 Best so far: ({best_solution['h1']}, {best_solution['h2']}, {best_solution['h3']}) | "
          f"lr={best_solution['lr']} | R² Test = {best_solution['r2_test']:.6f}")
    history.append((cycle+1, best_solution['r2_test']))

# === Final Results
print("\n📊 Final ABC-ANN Evaluation for SeismicFoS")
print(f"Best Hyperparameters: ({best_solution['h1']}, {best_solution['h2']}, {best_solution['h3']}, {best_solution['lr']})")
print(f"✅ R² (Train): {best_solution['r2_train']:.6f}")
print(f"✅ R² (Test):  {best_solution['r2_test']:.6f}")
print(f"📉 RMSE (Test): {best_solution['rmse']:.6f}")
print(f"📉 MAE (Test):  {best_solution['mae']:.6f}")
print(f"📉 MSE (Test):  {best_solution['mse']:.6f}")
print(f"📉 RMSE (Train): {best_solution['rmse_train']:.6f}")
print(f"📉 MAE (Train):  {best_solution['mae_train']:.6f}")
print(f"📉 MSE (Train):  {best_solution['mse_train']:.6f}")

# === Radar Plot
metrics_labels = ["R2_Train", "R2_Test", "RMSE_Test", "MAE_Test", "MSE_Test", "RMSE_Train", "MAE_Train", "MSE_Train"]
metrics_values = [
    best_solution["r2_train"],
    best_solution["r2_test"],
    best_solution["rmse"],
    best_solution["mae"],
    best_solution["mse"],
    best_solution["rmse_train"],
    best_solution["mae_train"],
    best_solution["mse_train"]
]

# Normalize for radar plot
max_vals = [1.0, 1.0, max(metrics_values[2:]), max(metrics_values[2:]), max(metrics_values[2:]),
            max(metrics_values[5:]), max(metrics_values[5:]), max(metrics_values[5:])]
norm_values = [v / m for v, m in zip(metrics_values, max_vals)]

# Create radar chart
fig = go.Figure()
fig.add_trace(go.Scatterpolar(
    r=norm_values + [norm_values[0]],
    theta=metrics_labels + [metrics_labels[0]],
    fill='toself',
    name='ABC-ANN SeismicFoS',
    line=dict(color='royalblue')
))
fig.update_layout(
    title="ABC-ANN Performance Radar Plot (SeismicFoS)",
    polar=dict(radialaxis=dict(visible=True, range=[0, 1])),
    showlegend=True
)

# Save radar plot
plot_path = os.path.join(output_dir, "ABC_ANN_SeismicFoS_RadarPlot.html")
fig.write_html(plot_path)
print(f"\n📁 Radar Plot saved to: {plot_path}")



🔄 Cycle 1
🐝 Best so far: (97, 30, 19) | lr=0.00744 | R² Test = 0.845865

🔄 Cycle 2
🐝 Best so far: (41, 107, 119) | lr=0.00704 | R² Test = 0.885254

🔄 Cycle 3
🐝 Best so far: (41, 107, 119) | lr=0.00704 | R² Test = 0.885254

🔄 Cycle 4
🐝 Best so far: (41, 107, 119) | lr=0.00704 | R² Test = 0.885254

🔄 Cycle 5
🐝 Best so far: (41, 107, 119) | lr=0.00704 | R² Test = 0.885254

🔄 Cycle 6
🐝 Best so far: (41, 107, 119) | lr=0.00704 | R² Test = 0.885254

🔄 Cycle 7
🐝 Best so far: (78, 18, 30) | lr=0.0093 | R² Test = 0.885565

🔄 Cycle 8
🐝 Best so far: (24, 16, 84) | lr=0.00768 | R² Test = 0.896037

🔄 Cycle 9
🐝 Best so far: (24, 16, 84) | lr=0.00768 | R² Test = 0.896037

🔄 Cycle 10
🐝 Best so far: (24, 16, 84) | lr=0.00768 | R² Test = 0.896037

📊 Final ABC-ANN Evaluation for SeismicFoS
Best Hyperparameters: (24, 16, 84, 0.00768)
✅ R² (Train): 0.983316
✅ R² (Test):  0.896037
📉 RMSE (Test): 0.302740
📉 MAE (Test):  0.192714
📉 MSE (Test):  0.091652
📉 RMSE (Train): 0.112807
📉 MAE (Train):  0.081925
📉 MSE